Traceback (most recent call last):
  File "inverted_pendulum_kinematics_cleanup.py", line 1, in <module>
    from vpython import *
ModuleNotFoundError: No module named 'vpython'


In [1]:
!conda install -y vpython control slycot

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - control
    - slycot
    - vpython


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aiohttp-3.6.2              |   py38h516909a_0         630 KB  conda-forge
    async-timeout-3.0.1        |          py_1000          11 KB  conda-forge
    autobahn-20.7.1            |             py_0         740 KB  conda-forge
    control-0.8.3              |           py38_1         351 KB  conda-forge
    dbus-1.13.6                |       he372182_0         602 KB  conda-forge
    expat-2.2.9                |       he1b5a44_2         191 KB  conda-forge
    gst-plugins-base-1.14.5    |       h0935bb2_2         6.8 MB  conda-forge
    gstreamer-1.14.5           |       h36ae1b5_2         4.5 MB  conda-forge
    jupyter-1.0.0              |             py_2           4 KB  conda-forge
   

In [ ]:
!python inverted_pendulum_kinematics_cleanup.py

Rank is 6
[[-356.59311628  -59.31996526   15.86393468   -3.0958639   -31.6227766
    -7.59638717]]
[[-2.54930585e+01 -7.28036658e+00  4.79218663e-01 -3.20213248e-03
  -1.00000000e+00 -2.60718140e-01]]
100%|███████████████████████████████████████| 1000/1000 [00:18<00:00, 55.08it/s]
